<a href="https://colab.research.google.com/github/Marcin19721205/BDCaseStudy/blob/main/BDCSZad3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split

In [2]:
path = "sample_data/CharlesBookClub.csv"
df = pd.read_csv(path)

print(df.shape)
print(df.head())

(4000, 24)
   Seq#  ID#  Gender    M   R  F  FirstPurch  ChildBks  YouthBks  CookBks  \
0     1   25       1  297  14  2          22         0         1        1   
1     2   29       0  128   8  2          10         0         0        0   
2     3   46       1  138  22  7          56         2         1        2   
3     4   47       1  228   2  1           2         0         0        0   
4     5   51       1  257  10  1          10         0         0        0   

   ...  ItalCook  ItalAtlas  ItalArt  Florence  Related.Purchase  Mcode  \
0  ...         0          0        0         0                 0      5   
1  ...         0          0        0         0                 0      4   
2  ...         1          0        0         0                 2      4   
3  ...         0          0        0         0                 0      5   
4  ...         0          0        0         0                 0      5   

   Rcode  Fcode  Yes_Florence  No_Florence  
0      4      2             0 

Podział danych

In [3]:
# split 60/40 train/test (shuffle, random_state=123)
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.4, random_state=123, shuffle=True)
train_df.shape, test_df.shape


((2400, 24), (1600, 24))

In [4]:
# wizualizacje poglądowe
import plotly.express as px

# rozkłady R/F/M (hist)
px.histogram(train_df, x="R", nbins=80, title="TRAIN — Recency (R) distribution", template="plotly_dark").show()
px.histogram(train_df, x="F", nbins=80, title="TRAIN — Frequency (F) distribution", template="plotly_dark").show()
px.histogram(train_df, x="M", nbins=80, title="TRAIN — Monetary (M) distribution", template="plotly_dark").show()

# rozkłady R/F/M (box)
px.box(train_df, y="R", title="TRAIN — Recency (R) boxplot", template="plotly_dark").show()
px.box(train_df, y="F", title="TRAIN — Frequency (F) boxplot", template="plotly_dark").show()
px.box(train_df, y="M", title="TRAIN — Monetary (M) boxplot", template="plotly_dark").show()

# Florence vs R/F/M (box)
px.box(train_df, x="Florence", y="R", title="TRAIN — R by Florence (0/1)", template="plotly_dark").show()
px.box(train_df, x="Florence", y="F", title="TRAIN — F by Florence (0/1)", template="plotly_dark").show()
px.box(train_df, x="Florence", y="M", title="TRAIN — M by Florence (0/1)", template="plotly_dark").show()

# korelacje (heatmap)
corr = train_df.drop(columns=[c for c in ["Seq#", "ID#"] if c in train_df.columns]).corr()  # bez ID
px.imshow(corr, title="TRAIN — Correlation heatmap (numeric predictors)", template="plotly_dark").show()

# scatter separacji
px.scatter(train_df, x="R", y="M", color="Florence", title="TRAIN — R vs M colored by Florence", template="plotly_dark").show()
px.scatter(train_df, x="F", y="M", color="Florence", title="TRAIN — F vs M colored by Florence", template="plotly_dark").show()


Przygotowanie danych pod K-najbliższych sąsasiadów

In [6]:
#przygotowanie cech pod KNN (RFM i ALL) + skalowanie (fit na train)
from sklearn.preprocessing import StandardScaler

y_train = train_df["Florence"]  # target train
y_test  = test_df["Florence"]   # target test

# KNN-RFM
X_train_rfm = train_df[["R", "F", "M"]]  # cechy train
X_test_rfm  = test_df[["R", "F", "M"]]   # cechy test

scaler_rfm = StandardScaler()  # scaler RFM
X_train_rfm_std = scaler_rfm.fit_transform(X_train_rfm)  # fit+transform train
X_test_rfm_std  = scaler_rfm.transform(X_test_rfm)       # transform test

# KNN-ALL (bez ID i duplikatów targetu)
drop_cols = [c for c in ["Florence", "Seq#", "ID#", "Yes_Florence", "No_Florence"] if c in train_df.columns]  # drop
X_train_all = train_df.drop(columns=drop_cols)  # cechy train
X_test_all  = test_df.drop(columns=drop_cols)   # cechy test

scaler_all = StandardScaler()  # scaler ALL
X_train_all_std = scaler_all.fit_transform(X_train_all)  # fit+transform train
X_test_all_std  = scaler_all.transform(X_test_all)       # transform test

print(X_train_rfm_std.shape, X_train_all_std.shape)


(2400, 3) (2400, 19)


In [13]:
#dobór k dla KNN (RFM i ALL) po ROC-AUC na TEST (ranking), k nieparzyste 1..25
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score

k_list = list(range(1, 26, 2))  # 1..25 nieparzyste

rows = []
for k in k_list:
    # RFM
    knn_rfm = KNeighborsClassifier(n_neighbors=k)  # model
    knn_rfm.fit(X_train_rfm_std, y_train)  # fit
    proba_rfm = knn_rfm.predict_proba(X_test_rfm_std)[:, 1]  # P(y=1)
    auc_rfm = roc_auc_score(y_test, proba_rfm)  # ROC-AUC (TEST)

    # ALL
    knn_all = KNeighborsClassifier(n_neighbors=k)  # model
    knn_all.fit(X_train_all_std, y_train)  # fit
    proba_all = knn_all.predict_proba(X_test_all_std)[:, 1]  # P(y=1)
    auc_all = roc_auc_score(y_test, proba_all)  # ROC-AUC (TEST)

    rows.append({"k": k, "auc_rfm": auc_rfm, "auc_all": auc_all})  # zapis

k_grid = pd.DataFrame(rows).sort_values("k").reset_index(drop=True)  # tabela wyników


best_k_rfm = int(k_grid.loc[k_grid["auc_rfm"].idxmax(), "k"])  # najlepsze k dla RFM
best_k_all = int(k_grid.loc[k_grid["auc_all"].idxmax(), "k"])  # najlepsze k dla ALL

print(k_grid)
print("-----------------best--------------------")
print(best_k_rfm, best_k_all)


     k   auc_rfm   auc_all
0    1  0.507718  0.527289
1    3  0.518210  0.571481
2    5  0.515542  0.569403
3    7  0.538977  0.584665
4    9  0.548969  0.590905
5   11  0.536672  0.595301
6   13  0.541444  0.600301
7   15  0.548917  0.597575
8   17  0.553959  0.582491
9   19  0.555649  0.587402
10  21  0.560026  0.584576
11  23  0.554040  0.584592
12  25  0.548234  0.583336
-----------------best--------------------
21 13


Trening

In [14]:
#trening KNN (RFM i ALL) na best_k + proba na TEST (P(Florence=1))
from sklearn.neighbors import KNeighborsClassifier

# RFM
knn_rfm = KNeighborsClassifier(n_neighbors=best_k_rfm)  # model RFM
knn_rfm.fit(X_train_rfm_std, y_train)  # trening
proba_test_knn_rfm = knn_rfm.predict_proba(X_test_rfm_std)[:, 1]  # P(y=1) na test

# ALL
knn_all = KNeighborsClassifier(n_neighbors=best_k_all)  # model ALL
knn_all.fit(X_train_all_std, y_train)  # trening
proba_test_knn_all = knn_all.predict_proba(X_test_all_std)[:, 1]  # P(y=1) na test

proba_test_knn_rfm[:10], proba_test_knn_all[:10]


(array([0.04761905, 0.0952381 , 0.0952381 , 0.        , 0.04761905,
        0.0952381 , 0.04761905, 0.04761905, 0.        , 0.19047619]),
 array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.07692308, 0.        , 0.07692308, 0.07692308]))

In [17]:
# fcumulative gains chart (TEST) dla KNN-RFM i KNN-ALL + linia losowa
import numpy as np
import pandas as pd
import plotly.express as px

def gains_curve(y_true, proba, model_name):  # helper
    t = pd.DataFrame({"y": y_true.values, "proba": proba})  # tabela
    t = t.sort_values("proba", ascending=False).reset_index(drop=True)  # sort malejąco
    t["cum_pop"] = (np.arange(len(t)) + 1) / len(t)  # % populacji
    total_pos = t["y"].sum()  # pozytywy
    t["cum_tp"] = t["y"].cumsum()  # kumulacja TP
    t["cum_gain"] = (t["cum_tp"] / total_pos) if total_pos > 0 else 0.0  # % przechwyconych pozytywów
    out = t[["cum_pop", "cum_gain"]].copy()  # krzywa
    out["model"] = model_name  # tag
    return out

g_knn_rfm = gains_curve(y_test, proba_test_knn_rfm, f"KNN — R,F,M only (k={best_k_rfm})")  # RFM
g_knn_all = gains_curve(y_test, proba_test_knn_all, f"KNN — All predictors (k={best_k_all})")  # ALL

g_rand = pd.DataFrame({"cum_pop": np.linspace(0, 1, 101), "cum_gain": np.linspace(0, 1, 101), "model": "Random (baseline)"})  # losowa

g_long = pd.concat([g_knn_rfm, g_knn_all, g_rand], ignore_index=True)  # long

fig = px.line(
    g_long,
    x="cum_pop",
    y="cum_gain",
    color="model",
    title="Skumulowany wykres zyskow (TEST) — KNN models vs Randomowy",
    template="plotly_dark"
)
fig.update_layout(
    xaxis_title="Population fraction targeted (highest predicted probability → lowest)",
    yaxis_title="Cumulative gain = captured positives / total positives (TEST)"
)
fig.show()


In [18]:
#Lift chart (TEST) dla KNN-RFM i KNN-ALL; baseline = rr_overall_test; decyle 10%
import numpy as np
import pandas as pd
import plotly.express as px

baseline_test = y_test.mean()  # rr_overall_test

def lift_table(y_true, proba, model_name):  # helper
    t = pd.DataFrame({"y": y_true.values, "proba": proba})  # tabela
    t = t.sort_values("proba", ascending=False).reset_index(drop=True)  # sort malejąco
    t["decile"] = (np.floor(np.arange(len(t)) / (len(t) / 10)) + 1).astype(int)  # 1..10
    t["decile"] = t["decile"].clip(1, 10)  # safety
    out = (t.groupby("decile")
           .agg(customers=("y", "size"), purchases=("y", "sum"), rr_decile=("y", "mean"))
           .reset_index())
    out["baseline_rr_test"] = baseline_test  # baseline (TEST)
    out["lift"] = out["rr_decile"] / out["baseline_rr_test"]  # lift
    out["model"] = model_name  # tag
    return out

lift_knn_rfm = lift_table(y_test, proba_test_knn_rfm, f"KNN — R,F,M only (k={best_k_rfm})")  # RFM
lift_knn_all = lift_table(y_test, proba_test_knn_all, f"KNN — All predictors (k={best_k_all})")  # ALL
lift_long = pd.concat([lift_knn_rfm, lift_knn_all], ignore_index=True)  # long

fig = px.line(
    lift_long,
    x="decile",
    y="lift",
    color="model",
    markers=True,
    hover_data=["customers", "purchases", "rr_decile", "baseline_rr_test"],
    title="Lift Chart (TEST) — KNN models; Lift = rr_decile / rr_overall_test",
    template="plotly_dark"
)
fig.update_layout(
    xaxis_title="Decile (1 = highest predicted probability, 10 = lowest)",
    yaxis_title="Lift = response_rate(decile) / overall_response_rate(TEST)"
)
fig.show()

lift_knn_rfm, lift_knn_all


(   decile  customers  purchases  rr_decile  baseline_rr_test      lift  \
 0       1        160         19    0.11875           0.08125  1.461538   
 1       2        160         16    0.10000           0.08125  1.230769   
 2       3        160         11    0.06875           0.08125  0.846154   
 3       4        160         17    0.10625           0.08125  1.307692   
 4       5        160         16    0.10000           0.08125  1.230769   
 5       6        160          9    0.05625           0.08125  0.692308   
 6       7        160         11    0.06875           0.08125  0.846154   
 7       8        160         10    0.06250           0.08125  0.769231   
 8       9        160         11    0.06875           0.08125  0.846154   
 9      10        160         10    0.06250           0.08125  0.769231   
 
                      model  
 0  KNN — R,F,M only (k=21)  
 1  KNN — R,F,M only (k=21)  
 2  KNN — R,F,M only (k=21)  
 3  KNN — R,F,M only (k=21)  
 4  KNN — R,F,M only (k

In [21]:
# Confusion Matrix (TEST) dla KNN-RFM i KNN-ALL @ threshold=0.5 (Plotly dark)
from sklearn.metrics import confusion_matrix

thr = 0.5  # próg

y_pred_knn_rfm = (proba_test_knn_rfm >= thr).astype(int)  # klasy RFM
y_pred_knn_all = (proba_test_knn_all >= thr).astype(int)  # klasy ALL

cm_knn_rfm = confusion_matrix(y_test, y_pred_knn_rfm)  # [[TN,FP],[FN,TP]]
cm_knn_all = confusion_matrix(y_test, y_pred_knn_all)  # [[TN,FP],[FN,TP]]

labels = ["0 (no)", "1 (yes)"]  # klasy
cm_rfm_df = pd.DataFrame(cm_knn_rfm, index=[f"Actual {l}" for l in labels], columns=[f"Pred {l}" for l in labels])  # df
cm_all_df = pd.DataFrame(cm_knn_all, index=[f"Actual {l}" for l in labels], columns=[f"Pred {l}" for l in labels])  # df

px.imshow(cm_rfm_df, text_auto=True, title=f"Confusion Matrix (TEST) — KNN (R,F,M only) k={best_k_rfm} @ thr=0.5", template="plotly_dark").show()
px.imshow(cm_all_df, text_auto=True, title=f"Confusion Matrix (TEST) — KNN (All predictors) k={best_k_all} @ thr=0.5", template="plotly_dark").show()
